In [1]:
from qcnn.quantum.operation import C2Q
import pennylane as qml
from pennylane import numpy as np

device = qml.device('default.qubit', wires=range(3))

theta = np.ones(7)
phi = np.ones(7)
t = np.ones(7)

@qml.qnode(device)
def circuit():
    C2Q(theta, phi, t, wires=range(3), angles=True)
    return qml.state()
    
circuit()

tensor([ 0.67587122-9.13476639e-18j,  0.19949589+3.10696442e-01j,
         0.19949589+3.10696442e-01j, -0.08394154+1.83415602e-01j,
         0.19949589+3.10696442e-01j, -0.08394154+1.83415602e-01j,
        -0.08394154+1.83415602e-01j, -0.10909263+1.55507768e-02j], requires_grad=True)

In [26]:
import torch
import numpy as np

i = 4
n = 2 ** (i + 1) - 1
params = list(range(n))

print(n, 2 ** i, )
num_phi = (len(params) + 1) // 4

print(params[-num_phi:])

t, theta, phi = params[:num_phi], params[num_phi:-num_phi], params[-num_phi:]

t, theta, phi

31 16
[23, 24, 25, 26, 27, 28, 29, 30]


([0, 1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
 [23, 24, 25, 26, 27, 28, 29, 30])

In [7]:
from pennylane.wires import Wires

Wires.all_wires(3)

TypeError: 'int' object is not iterable

In [7]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import Adam, SGD
from torch.nn import CrossEntropyLoss, MSELoss

from qcnn.ml.data import BinaryData
from qcnn.ml.data import image_transform, baseline_image_transform
from qcnn.ml.optimize import Optimizer
from qcnn.experiment import Experiment
from qcnn.qcnn import QCNN
from qcnn.cnn import CNN
from qcnn.file import save_dataframe_as_csv

print(CrossEntropyLoss())

CrossEntropyLoss()
